In [1]:
!pip install opencv-python pygmtools numpy scipy networkx pandas shapely bplustree svgpathtools drawsvg

In [2]:
import pandas as pd

import numpy as np

import scipy as sc
import scipy.io as scio # for loading .mat file
from scipy import linalg

from sklearn.decomposition import PCA as PCAdimReduc
from sklearn.feature_extraction import DictVectorizer

import networkx as nx

import pygmtools as pygm

import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch

import cv2

import shapely

import svgpathtools
import drawsvg as draw

from bplustree import BPlusTree
from bplustree.serializer import Serializer

import pickle

import functools
from itertools import product
from typing import Optional, Tuple, List
from collections import Counter

import struct

import os
import sys

import random

pygm.BACKEND = 'numpy' # set numpy as backend for pygmtools

# TOPOLOGY GRAPH / GEOMETRY EXTRACTION

In [ ]:
def load(filename):
    """
    Loads an image from the image library given its filename. 
    """
    img = cv2.imread(filename) 
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def display(img, text='test'):
    """
    Displays a provided image to the user, and wait for any input. Used for debugging.
    """
    imgplot = plt.imshow(img)

    plt.show()

In [ ]:
IMAGE_DIRECTORY = os.sep.join(['assets', 'png'])

def get_files(ext='png'):
    """
    Returns an iterator that yields the image file paths and their labels.
    """    
    def get_label(path):
        return path.split(os.sep)[-1]
    
    files = []
    
    for r, _, f in os.walk(IMAGE_DIRECTORY):
        for file in f:
            if file.endswith(f'.{ext}'):
                label = get_label(r)
                file_path = os.path.join(r, file)
                
                files.append((file_path, label))
                
    return files

In [ ]:
def load_images(files):
    """
    Returns an iterator that yields the image data and their labels.
    """
    for f, l in files:
        yield load(f), l
        
def extract_graphs(images):
    """
    Extract all graphs from given images.
    """
    i = 0
    graphs = []
    for img, l in images:
        graphs.append(extract_graph(img, l))
        print(i)
        i += 1
    return graphs

In [ ]:
def extract_graph(img, label) -> nx.Graph:
    """
    Extract topology and geometry graph from source (image, label) pair.
    
    Nodes signify visual units in the image. These include:
    - Strokes (lines, curves)
    - Shapes (circle, rectangle)
    
    Nodes contain data, including:
    - Shape information
    - Center of unit in image coordinates
    
    Edges specify spatial relationships between nodes, as per 
    Sousa & Fonseca. “Sketch-Based Retrieval of Drawings using Topological Proximity” :
    - Adjacency: two nodes intersect
    - Composition: a node is contained inside another
    """
    G = nx.Graph()
    
    # Get sketch contours
    contours, hierarchy = get_contours(img)
    
    # Use contour centroids to get node
    centroids = get_centroids(contours)
    
    # Use hull to derive adjacency relations
    hulls = get_hulls(contours)
    
    # Create nodes with centroid, perimeter, and area attributes
    for i, c in enumerate(centroids):
        if c != None and hulls[i] is not None:
            length, area = hulls[i].length, hulls[i].area
            G.add_node(i, position=c , length=length, area=area)
    
    # Create neighbor edges using convex hull intersection
    for i, j in product(G.nodes, repeat=2):
        h1, h2 = hulls[i], hulls[j]
        if i != j and h1 is not None and h2 is not None and h1.intersects(h2):
                G.add_edge(i, j, relation='neighbor')
                
    # Create parent edges using contour hierarchy tree
    for i in G.nodes:
        parent = hierarchy[0, i, -1]
        if parent > -1:
            G.add_edge(i, parent, relation='parent')
    
    # Add graph labels
    G.graph['label'] = label
    G.graph['positions'] = {i : np.array(c) for i, c in enumerate(centroids)}
    
    return G

In [ ]:
def get_contours(img) -> List[np.array]:
    blur = cv2.medianBlur(img, 7)

    ret, thresh = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     for i in range(len(contours)):
#         img2 = cv2.drawContours(img.copy(), contours, i, (0,255,0), 3)
#         display(img2)

    return contours, hierarchy

In [ ]:
def get_centroids(contours) -> List[Optional[Tuple[int, int]]]:
    """
    Computes the centroids of the given contours
    """
    def get_centroid(c):
        M = cv2.moments(c)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            return (cx, cy)
        return None
            
    return [get_centroid(c) for c in contours]

In [ ]:
def get_hulls(contours) -> List[Optional[shapely.Polygon]]:
    """
    Computes the convex hulls of the given contours, and returns a polygon object.
    """
    def to_polygon(hull):
        n, *_ = hull.shape
        if n > 3:
            return shapely.Polygon(np.reshape(hull, (n, 2)))
        return None
    
    return [to_polygon(cv2.convexHull(c)) for c in contours]

In [ ]:
def plot_graph(G):
    pos = G.graph['positions']
    plt.figure()
    nx.draw(
        G, pos, edge_color='black', width=1, linewidths=1,
        node_size=500, node_color='blue', alpha=0.5
    )
    nx.draw_networkx_edge_labels(
        G, pos,
        edge_labels=nx.get_edge_attributes(G,'relation'),
        font_color='red'
    )
    plt.axis('off')
    plt.show()

In [ ]:
img = load('assets/png/angel/189.png')
display(img)

G = extract_graph(img, 'angel')
plot_graph(G)

# DATABASE

In [ ]:
def dump_graphs(graphs):
    with open("graphs.p", 'wb') as f:
        pickle.dump(graphs, f)
    
def load_graphs() -> List[nx.Graph]:
    with open("graphs.p", 'rb') as f:
        return pickle.load(f)

In [ ]:
class NeighborBPlusTree(BPlusTree): 
    def get_neighbors(self, key, default={}) -> dict:
        with self._mem.read_transaction:
            node = self._search_in_tree(key, self._root_node)
            records = node.entries
            rv = {r.key: self._get_value_from_record(r) for r in records}
            assert isinstance(rv, dict)
            return rv
            
class DoubleSerializer(Serializer):
    __slots__ = []

    def serialize(self, obj: float, key_size: int) -> bytes:
        return struct.pack('d', obj)

    def deserialize(self, data: bytes) -> float:
        return struct.unpack('d', data)[0]

In [ ]:
def construct_database(graphs) -> BPlusTree:
    """
    Populates database with all graphs extracted from images, along with image index and label.
    
    Uses the algorithms described in Fonseca and Jorge "Indexing High-Dimensional Data for 
    Content-Based Retrieval in Large Databases".
    
    Constructs a B+tree that uses the index described in the paper to map labels, facilitates
    range queries and KNN queries. 
    
    The B+-tree maps the index to graph information, serialized to bytes.
    
    TO-DO: change from only encoding label to more graph features!
    """
    def construction_index(G):
        return perturbe(index(G))
    
    # Create database, with key_size of 8 bytes (C double), other parameters to be determined
    db = NeighborBPlusTree('db/labels.db', serializer=DoubleSerializer(), order=64, key_size=8)
    
    try:
        # Create iterator for all key/value pairs to be inserted into database
        iterator = sorted( 
            zip(
                # All keys
                map(construction_index, graphs),
                # All values (binary)
                map(lambda G: G.graph['label'].encode(), graphs)
            ),
            key=lambda t: t[0]
        )
        counts = Counter(map(lambda t: t[0], iterator))
        print(len(counts))
        assert len(counts) == 20_000

#         # Insert all key/value pairs into the database
        db.batch_insert(iterator)
#         for k, v in iterator:
#             db[k] = v

        # Flush
        db.checkpoint()
        return db        
        
    except:
        # DB unexpected error, close.
        erase_database(db)
        raise 

def erase_database(db):
    db.close()
    os.remove('db/labels.db')

In [ ]:
def query_database(db, query):
    """
    Returns set of candidate graphs from the database using a query graph.
    
    Uses the K-NN query from Fonseca and Jorge "Indexing High-Dimensional 
    Data for Content-Based Retrieval in Large Databases".
    """
    key = index(query)
    
    return db.get_neighbors(key)

In [ ]:
def index(graph):
    """
    Get the topology index of the graph, as described in 
    Sousa and Fonseca "Sketch-Based Retrieval of Drawings using Topological Proximity"
    and in Fonseca and Jorge "Indexing High-Dimensional Data for Content-Based Retrieval 
    in Large Databases".
    
    Is the norm of the array of absolute value eigenvalues of the graph's adjacency matrix.
    
    This is used as the index of a B+-tree to find graphs with similar topology.
    """
    # Get adjacency matrix of graph
    A = nx.to_numpy_array(graph)
    
    # Compute absolute values of eigenvalues 
    spectra = np.absolute(linalg.eigvals(A))
    
    # Get norm of eigenvalues (the reduction to 1 dimensions)
    norm = linalg.norm(spectra)
    
    return norm

In [ ]:
def perturbe(index):
    # Add small random perturbation to avoid collisions
    epsilon = (2*random.random() - 1) / 100000000
    return index + epsilon

In [ ]:
perturbe(index(G))

## Offline graph extraction

In [ ]:
# Extract graphs for each sketch in the dataset, offline processing step

paths = get_files()
print(len(paths))

images = load_images(paths)

graphs = extract_graphs(images)
print(len(graphs))

In [ ]:
# Dump the graphs to avoid this offline step later
dump_graphs(graphs)

## Database construction

In [3]:
# Load graphs from disk
graphs = load_graphs()

NameError: name 'load_graphs' is not defined

In [ ]:
# Construct database for online use
db = construct_database(graphs)

In [ ]:
# Test query
query = G

candidates = query_database(db, query)
print(candidates)

counts = Counter(candidates.values())
print(counts)

In [ ]:
db.close()

In [ ]:
a,b,c = encode(G)
print(a, b.shape, c.shape)

# GRAPH MATCHING

In [ ]:
def match(graph1, graph2) -> np.array:
    """
    Uses a graph match solver to find a match between two topological feature graphs.
    
    Uses the Quadratic Assignment Problem formulation of graph matching to encode graphs.
    
    The solver and parameters utilized are an implementation detail in this function.
    """
    K = build_affinity(graph1, graph2)
    X = pygm.rrwm(K, graph1.number_of_nodes(), graph2.number_of_nodes())
    return pygm.hungarian(X)

In [ ]:
def build_affinity(graph1, graph2) -> np.array:
    """
    Construct affinity matrix for matching QAP solver.
    """
    node1, edge1, conn1 = encode(graph1)
    node2, edge2, conn2 = encode(graph2)
    
    gaussian_aff = functools.partial(pygm.utils.gaussian_aff_fn, sigma=1) # set affinity function
    
    return pygm.utils.build_aff_mat(node1, edge1, conn1, node2, edge2, conn2, edge_aff_fn=gaussian_aff)

In [ ]:
def encode(graph) -> Tuple[np.array, np.array, np.array]:
    """
    Encode graph as a edge feature matrix, and node feature matrix, 
    connectivity matrix triple. This is fed to a QAP solver to match graphs.
    
    """
    # Extract array node features
    n_f = extract_node_features(graph)
    
    # Extract adjacency matrix edge features
    e_f = extract_edge_features(graph)
    
    # Derive connectivity matrix and edge features array from adjacency matrix edge features
    conn, edge = pygm.utils.dense_to_sparse(e_f)
    
    return n_f, edge, conn

In [ ]:
def extract_edge_features(graph) -> np.array:
    """
    Extract edge features in the form of an adjacency matrix with an extra feature vector axis.
    
    Can utilize any of the input graph's graph, node, and edge labels to derive features.
    
    Can utilize scikit-learn's feature extraction to vectorize discrete and continuous features:
    https://scikit-learn.org/stable/modules/feature_extraction.html
    """
    # Get adjacency matrix from graph
    A = nx.to_numpy_array(graph)
    
    # ----------------
    # Length feature
    # ----------------
    
    # Get a 'positions' graph attribute:
    pos = graph.graph['positions']
    
    # Convert position dictionary to numpy array
    pos_arr = position_array(pos)

    # Derive edge distances from adjacency matrix
    A = ((np.expand_dims(pos_arr, 1) - np.expand_dims(pos_arr, 2)) ** 2).sum(axis=0) * A
    A = (A / A.max()).astype(np.float32)
    
    return A

In [ ]:
def extract_node_features(graph) -> Optional[np.array]:
    """
    Extract node features in the form of an array of feature vectors.
    
    Can return None to indicate no node features.
    
    Can utilize any of the input graph's graph, node, and edge labels to derive features.
    
    Can utilize scikit-learn's feature extraction to vectorize discrete and continuous features:
    https://scikit-learn.org/stable/modules/feature_extraction.html
    """
    return None

In [ ]:
def position_array(pos):
    return np.array(list(pos.values())).T

In [ ]:
def plot_graph(G, color='k'):
    """
    Plots a given graph with a 'positions' attribute.
    """
    assert 'positions' in G.graph
    
    A = nx.to_numpy_array(G)
    
    pos = G.graph['positions']
    pts = position_array(pos)
    plt.scatter(pts[0], pts[1], c='w', edgecolors=color)

    for x, y in zip(np.nonzero(A)[0], np.nonzero(A)[1]):
        plt.plot((pts[0, x], pts[0, y]), (pts[1, x], pts[1, y]), color+'-')

In [ ]:
def plot_mapping(X, graph1, graph2):
    assert 'positions' in graph1.graph
    assert 'positions' in graph2.graph
    
    pos1 = graph1.graph['positions']
    pos2 = graph2.graph['positions']
    
    plt.figure(figsize=(8, 4))
    plt.suptitle('Image Matching Result by RRWM')

    ax1 = plt.subplot(1, 2, 1)
    plot_graph(graph1)

    ax2 = plt.subplot(1, 2, 2)
    plot_graph(graph2)

    pts1 = position_array(pos1)
    pts2 = position_array(pos2)
    
    for i in range(X.shape[0]):
        j = np.argmax(X[i]).item()
        con = ConnectionPatch(xyA=pts1[:, i], xyB=pts2[:, j], coordsA="data", coordsB="data",
                              axesA=ax1, axesB=ax2, color="red" if i != j else "green")
        plt.gca().add_artist(con)

In [ ]:
G = nx.tutte_graph()
G.graph['positions'] = nx.spring_layout(G)

In [ ]:
nx.draw(G, pos=G.graph['positions'])

In [ ]:
ax1 = plt.subplot(1, 2, 1)
plot_graph(G)

ax2 = plt.subplot(1, 2, 2)
plot_graph(G)

In [ ]:
X = match(G, G)
print(X.shape)
X

In [ ]:
plot_mapping(X, G, G)